In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,roc_curve,auc,accuracy_score

pd.options.display.html.table_schema=True
pd.options.display.max_rows=None
pd.options.display.max_columns=None
# %matplotlib inline

In [2]:
data=pd.read_excel('111.xlsx',sheet_name='Лист2')
X=np.array(data.drop('Событие', axis=1))
y=np.array(data['Событие'])

In [3]:
rand_forest=RandomForestClassifier(random_state=0, n_estimators=50, max_depth=5).fit(X, y)

In [4]:
importances = [{data.columns[index]: x} for index,x in enumerate(rand_forest.feature_importances_) if x>0.05]
importances_ = []
for col in importances:
    importances_.append( list(col.keys())[0] )
    print(col)

{'Покраснение': 0.0613548482753228}
{'выделение из глаз – характер отделяемого': 0.05888240097149655}
{'неприятные ощущения в глазу – присутствие – имеется': 0.07650216229376981}
{'КГЯ – изменения': 0.05451842715016816}
{'конъюнктива ВиН век – изменения – отек': 0.05606512035082479}
{'конъюнктива ВиН век -  характер отделяемого': 0.060100686245273496}


In [5]:
print(rand_forest.score(X,y))
y_pred=rand_forest.predict(X)
pd.crosstab(y,y_pred)

0.9310344827586207


col_0,0,1,2,4
row_0,,,,
0,16,0,0,0
1,1,5,0,0
2,0,0,3,0
3,0,1,0,0
4,0,0,0,3


In [6]:
New_X=data[[list(_.items())[0][0] for _ in importances]]
rand_forest1=RandomForestClassifier(random_state=0, n_estimators=40, max_depth=5).fit(New_X, y)
print(rand_forest1.score(New_X,y))

0.896551724137931


In [7]:
from collections import OrderedDict
interact_data = OrderedDict()
copied = pd.DataFrame(data, columns=importances_)
# copied = copied.drop('Событие', 1)
for col in copied:
    col_data = data[col].unique()
    interact_data[col] =(col_data.min(), col_data.max())

In [8]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

def make_preset(**kwargs):
    predicts = list(kwargs.values())
    predicts = [np.asanyarray(predicts)]
    
    predicted = rand_forest1.predict([predicts[0]])
    return f'Предсказанное событие {predicted[0]}'
    
list_widgets={}

for name in interact_data.keys():
    value = interact_data[name]
    list_widgets[name] = widgets.IntSlider(value=value[0] ,min=value[0],max=value[1],description=name)

interact(make_preset, **list_widgets);

interactive(children=(IntSlider(value=0, description='Покраснение', max=2), IntSlider(value=0, description='вы…

In [9]:
[ print(_) for _ in interact_data.keys()]

Покраснение
выделение из глаз – характер отделяемого
неприятные ощущения в глазу – присутствие – имеется
КГЯ – изменения
конъюнктива ВиН век – изменения – отек
конъюнктива ВиН век -  характер отделяемого


[None, None, None, None, None, None]

In [10]:
from sklearn.tree import export_graphviz

# Export as dot file
export_graphviz(rand_forest1.estimators_[39], 
                out_file='tree.dot', 
                feature_names = [ _ for _ in interact_data.keys()],
                class_names = ['1', '2', '3', '4', '5'],
                rounded = True, proportion = False, 
                precision = 3, filled = True)

In [11]:
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

0

In [15]:
import requests
import json
requests.get(f'https://iacpaas.dvo.ru/api/service/run/4640883221581498862')

<Response [403]>

In [92]:
import_link = 'https://iacpaas.dvo.ru/api/fund/structure/export/user-item?path=%D0%A1%D0%B5%D1%80%D0%B2%D0%B8%D1%81%20%D0%B4%D0%B8%D0%B0%D0%B3%D0%BD%D0%BE%D1%81%D1%82%D0%B8%D0%BA%D0%B8%20%D0%B1%D0%B5%D0%B7%20%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D1%84%D0%B5%D0%B9%D1%81%D0%B0%2F%D0%90%D1%80%D1%85%D0%B8%D0%B2%20%D1%81%20%D0%BE%D0%B4%D0%BD%D0%BE%D0%B9%20%D0%98%D0%91&json-type=universal'
#TODO: APP TOKEN HERE
api_key = '6cf60d216c5b1b32ebfbbb5492c5a1b7'
response = requests.get(import_link, headers={"accept": "application/json", "X-API-KEY": api_key})
result_json = json.loads(response._content.decode('utf-8'))

In [93]:
json_data = json.loads(result_json['data'])
# json_data['title'] = 'test'
# json_data['path'] = '/Сервис диагностики без интерфейса/test'
# json_data['name'] = 'test'
# json_data['owner_id'] = 216
# result_json['data'] = json_data
# result_json['filename'] = 'test'

# json_data['successors'][1]['successors'][0]['successors'][1]['successors'][0]['value'] = 99
# json_data['successors'][1]['successors'][0]['successors'][1]['successors'][1]['value'] = 2021
# json_data['successors'][1]['successors'][0]['successors'][0]['successors'][0]['value'] = 'не опознано'
# json_data['successors'][1]['successors'][1]['successors'][0]['successors'][0]['successors'][0]['successors'][0]['successors'].append(
# {'id': 73804718014540,
#   'name': 'Качественные значения',
#   'type': 'НЕТЕРМИНАЛ',
#   'meta': 'Качественные значения',
#   'successors': [{'id': 73804718014540,
#     'value': 'мокрое',
#     'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ',
#     'valtype': 'STRING',
#     'meta': 'значение',
# #     'original': 'margaret@iacp.dvo.ru/Мой Фонд/Новая база наблюдений/База медицинской терминологии и наблюдений 2020$/НАБЛЮДЕНИЯ/ПРИЗНАКИ/Жалобы/Общие/Выделение из полости носа/Составной признак/Характер отделяемого/Тип возможных значений/Качественные значения/слизистое;',
#     'rcomment': '',
#     'comment': ''}],
#   'rcomment': '',
#   'comment': ''}
# )

# json_data['successors'][1]['successors'][1]['successors'][0]['successors'][0]['successors'][0]['successors'][0]['successors'].append(
# {'id': 73804718014540,
#   'name': 'Качественные значения',
#   'type': 'НЕТЕРМИНАЛ',
#   'meta': 'Качественные значения',
#   'successors': [{'id': 73804718014540,
#     'value': 'сухое',
#     'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ',
#     'valtype': 'STRING',
#     'meta': 'значение',
# #     'original': 'margaret@iacp.dvo.ru/Мой Фонд/Новая база наблюдений/База медицинской терминологии и наблюдений 2020$/НАБЛЮДЕНИЯ/ПРИЗНАКИ/Жалобы/Общие/Выделение из полости носа/Составной признак/Характер отделяемого/Тип возможных значений/Качественные значения/слизистое;',
#     'rcomment': '',
#     'comment': ''}],
#   'rcomment': '',
#   'comment': ''}
# )

# json_data['successors'][1]['successors'][1]['successors'][0]['successors'][0]['successors'][0]['successors'][0]['successors'].append(
# {'id': 73804718014540,
#   'name': 'Качественные значения',
#   'type': 'НЕТЕРМИНАЛ',
#   'meta': 'Качественные значения',
#   'successors': [{'id': 73804718014540,
#     'value': 'не определился',
#     'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ',
#     'valtype': 'STRING',
#     'meta': 'значение',
# #     'original': 'margaret@iacp.dvo.ru/Мой Фонд/Новая база наблюдений/База медицинской терминологии и наблюдений 2020$/НАБЛЮДЕНИЯ/ПРИЗНАКИ/Жалобы/Общие/Выделение из полости носа/Составной признак/Характер отделяемого/Тип возможных значений/Качественные значения/слизистое;',
#     'rcomment': '',
#     'comment': ''}],
#   'rcomment': '',
#   'comment': ''}
# )

In [94]:
json_data['successors'][0]['successors'][1]['successors'].append(
{'id': 73804718015008, 'name': ' Резь в глазу', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Признак', 'successors': [{'id': 73804718015012, 'name': 'Составные значения', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Составные значения', 'successors': [{'id': 73804718015016, 'name': 'Присутствие', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Характеристика', 'successors': [{'id': 73804718015020, 'name': 'Выбор типа значений', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Выбор типа значений', 'successors': [{'id': 73804718015024, 'name': 'Качественные значения', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Качественные значения', 'successors': [{'id': 73804718015030, 'value': 'имеется', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': 'значение', 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}
)

json_data['successors'][0]['successors'][1]['successors'].append(
{'id': 73804718015008, 'name': 'Покраснение глаза', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Признак', 'successors': [{'id': 73804718015012, 'name': 'Составные значения', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Составные значения', 'successors': [{'id': 73804718015016, 'name': 'Присутствие', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Характеристика', 'successors': [{'id': 73804718015020, 'name': 'Выбор типа значений', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Выбор типа значений', 'successors': [{'id': 73804718015024, 'name': 'Качественные значения', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Качественные значения', 'successors': [{'id': 73804718015030, 'value': 'имеется', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': 'значение', 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}
)

json_data['successors'][0]['successors'][1]['successors'].append(
{'id': 73804718015008, 'name': 'Боль в глазу', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Признак', 'successors': [{'id': 73804718015012, 'name': 'Составные значения', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Составные значения', 'successors': [{'id': 73804718015016, 'name': 'Иррадиация', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Характеристика', 'successors': [{'id': 73804718015020, 'name': 'Выбор типа значений', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Выбор типа значений', 'successors': [{'id': 73804718015024, 'name': 'Качественные значения', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Качественные значения', 'successors': [{'id': 73804718015030, 'value': 'половина головы', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': 'значение', 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}], 'rcomment': '', 'comment': ''}
)

In [95]:
resp_post = requests.post('https://iacpaas.dvo.ru/api/fund/structure/import', headers={"Content-Type": "application/json", "X-API-KEY": api_key, "accept": "application/json"}, data=json.dumps({
    'path': '/Сервис диагностики без интерфейса',
    'json': json.dumps(json_data),
    'clearIfExists': True
}))
print(json.loads(resp_post._content.decode('utf-8')))




{'error': None, 'explanation': None, 'success': True, 'platformUnavailable': False, 'explanationSafe': None}


In [96]:
response_ = requests.get('https://iacpaas.dvo.ru/api/service/run/4640197744801057262', headers={"accept": "application/json", "X-API-KEY": api_key})
print(json.loads(response_._content))

{'error': None, 'explanation': None, 'success': True, 'platformUnavailable': False, 'tpir': '', 'attachResult': None, 'explanationSafe': None}


In [ ]:
json_data